In [ ]:
import numpy as np 
import pandas as pd 
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import albumentations as A

from sklearn.metrics import f1_score, classification_report
import pickle
from sklearn.model_selection import StratifiedKFold
from numpy import array
from random import shuffle, seed

In [ ]:
list_paths = []
for subdir, dirs, files in os.walk("../input"):
    for file in files:
        filepath = subdir + os.sep + file
        list_paths.append(filepath)
        
list_train = [filepath for filepath in list_paths if "train/" in filepath]
seed(420)
shuffle(list_train)
list_test = [filepath for filepath in list_paths if "test/" in filepath]

In [ ]:
list_train[:5]

In [ ]:
def get_class_from_path(filepath):
    return os.path.dirname(filepath).split(os.sep)[-1]

In [ ]:
labels = [get_class_from_path(filepath) for filepath in list_train]

In [ ]:
labels[:5]

In [ ]:
train_data = pd.DataFrame(labels, columns=['class'])
train_data['path'] = list_train
train_data.head()

In [ ]:
train_data['class'].value_counts().sort_values().plot(kind='bar')

In [ ]:
X = train_data['path']
y = train_data['class']

In [ ]:
splitter = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
splits = list(splitter.split(X=X, y=y))

In [ ]:
y.shape

In [ ]:
y = pd.get_dummies(y)
y.head()

In [ ]:
dict_map = {"0": "HTC-1-M7", 
            "1": "LG-Nexus-5x", 
            "2": "Motorola-Droid-Maxx", 
            "3": "Motorola-Nexus-6", 
            "4": "Motorola-X", 
            "5": "Samsung-Galaxy-Note3",
            "6": "Samsung-Galaxy-S4",
            "7": "Sony-NEX-7",
            "8": "iPhone-4s",
            "9": "iPhone-6"}

In [ ]:
def open_image(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [ ]:
SHAPE = 512

train_augmentations = A.Compose([A.RandomCrop(height=SHAPE,width=SHAPE),
                                 A.RandomGamma(gamma_limit=(80, 120), p=0.8),
                                 A.JpegCompression(quality_lower=70, quality_upper=90, p=0.9),
                                 A.GridDistortion(interpolation=cv2.INTER_CUBIC, p=0.9)
                                 ])

teste_augmentations = A.Compose([A.CenterCrop(height=SHAPE,width=SHAPE)                              
                                 ])

In [ ]:
class CameraDataset(tf.keras.utils.Sequence):
    def __init__(self, X_set, y_set, batch_size, augmenter=None, test=False, *args, **kwargs):
        
        self.batch_size = batch_size
        self.x_set = X_set
        self.y_set = y_set
        self.test = test
        self.augmenter = augmenter
        
    def __len__(self):
        return int(len(self.x_set) / self.batch_size)
    
    
    def __getitem__(self, index):
        X = self.x_set[index * self.batch_size : (index + 1) * self.batch_size]        
        y = self.y_set[index * self.batch_size : (index + 1) * self.batch_size]
        
        X = [(self.augmenter(image=open_image(x))['image']) for x in X]
        
        if self.test:
            return np.array(X)
        
        return np.array(X), y.values

In [ ]:
train_dataset = CameraDataset(X, y, batch_size=8, augmenter=train_augmentations)

In [ ]:
x_set, y_set = train_dataset.__getitem__(50)

In [ ]:
x_set[0].shape

In [ ]:
y_set[0]

In [ ]:
plt.imshow(x_set[3])

In [ ]:
def build_model():
    base_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, input_shape=[SHAPE, SHAPE, 3])
    base_model.trainable = True
    inputs =tf.keras.layers.Input(shape=(SHAPE,SHAPE,3))
    x = tf.keras.applications.densenet.preprocess_input(inputs)
    x = base_model(x, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model =tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    model.summary() 
    return model

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

train_dataset = CameraDataset(X_train, y_train, batch_size=10,augmenter=train_augmentations)
val_dataset = CameraDataset(X_test, y_test, batch_size=10,augmenter=teste_augmentations)

In [ ]:
model = build_model()

In [ ]:
file_path = "weights.best.hdf5"

checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, monitor="val_accuracy", save_best_only=True, mode='max')

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.9, patience=2, min_lr=1e-6, mode="max", verbose=True)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, mode="max", verbose=True)

callbacks_list = [checkpoint, reduce_lr, early_stopping]

history = model.fit(train_dataset, validation_data=val_dataset, epochs=100, batch_size=10, callbacks=callbacks_list)

In [ ]:
sample_submission = pd.read_csv('../input/sp-society-camera-model-identification/sample_submission.csv')
sample_submission.head()

In [ ]:
X_test = '../input/sp-society-camera-model-identification/test/test/' + sample_submission['fname']

In [ ]:
X_test[:5]

In [ ]:
def read_and_array(filepath):
    im_array = np.array(Image.open((filepath)), dtype="uint8")
    pil_im = Image.fromarray(im_array)
    new_array = np.array(pil_im)
    return new_array

In [ ]:
file_path = "weights.best.hdf5"
model = tf.keras.models.load_model(file_path)
test_dataset = CameraDataset(X_test, sample_submission['camera'], test=True, augmenter=teste_augmentations, batch_size=10, shuffle=False)
pred_mean = model.predict(test_dataset)
# X_test = np.array([read_and_array(filepath) for filepath in X_test])
# pred_mean = model.predict(X_test)

In [ ]:
labels_test=[]
for item in pred_mean.argmax(axis=1):
    labels_test.append(dict_map[str(item)])

In [ ]:
pred_mean.argmax(axis=1)[:10]

In [ ]:
labels_test[:10]

In [ ]:
sample_submission['camera'] = labels_test
sample_submission.head()

In [ ]:
  sample_submission['camera'].value_counts()

In [ ]:
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
sample_submission['camera'].value_counts().sort_values().plot(kind='bar')